In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
import pickle
import os


def log(msg):
    print(msg, '\n')


def createFolders():
    for breed in list(classDic.keys()):
        # Breeds
        trainPath = os.path.join('./breeds/train/' + breed)
        if not os.path.exists(trainPath):
            os.mkdir('./breeds/train/' + breed)

        testPath = os.path.join('./breeds/test/' + breed)
        if not os.path.exists(testPath):
            os.mkdir('./breeds/test/' + breed)

        # Dogs
        trainPath = os.path.join('./species/train/dog')
        if not os.path.exists(trainPath):
            os.mkdir('./species/train/dog')

        testPath = os.path.join('./species/test/dog')
        if not os.path.exists(testPath):
            os.mkdir('./species/test/dog')

        # Cats
        trainPath = os.path.join('./species/train/cat')
        if not os.path.exists(trainPath):
            os.mkdir('./species/train/cat')

        testPath = os.path.join('./species/test/cat')
        if not os.path.exists(testPath):
            os.mkdir('./species/test/cat')


def splitBreeds():
    for imgName in list(filesDic.keys()):
        img = keras.preprocessing.image.load_img('images/' + imgName)
        foldTrain = filesDic.get(imgName).get('foldTrain')

        keras.preprocessing.image.save_img(
            './breeds/' + ('train/' if foldTrain else 'test/') + filesDic.get(imgName).get('breed') + '/' + imgName, img)


def splitSpecies():
    for imgName in list(filesDic.keys()):
        img = keras.preprocessing.image.load_img('images/' + imgName)
        foldTrain = filesDic.get(imgName).get('foldTrain')

        keras.preprocessing.image.save_img(
            './species/' + ('train/' if foldTrain else 'test/') + filesDic.get(imgName).get('species') + '/' + imgName, img)


D = dict(pickle.load(open('Oxford-IIIT-Pet_Dics.p', 'rb')))
imageNum = 7390

DKeys = list(D.keys())
# log(DKeys)

classDic = dict(D.get('classDic'))
# log(classDic)

filesDic = dict(D.get('filesDic'))
# log(list(filesDic.keys()))
filesDicValues = list(filesDic.values())
log(filesDicValues[-1:])

# createFolders()
# splitBreeds()
# splitSpecies()

# log(D)


# CNN network from scratch

In [ ]:
cnn = models.Sequential()

cnn.add(layers.Conv2D(16, (5, 5), activation='relu',
                      input_shape=(200, 200, 1), padding="same"))
cnn.add(layers.Conv2D(16, (5, 5), activation='relu'))
cnn.add(layers.AveragePooling2D((2, 2)))
cnn.add(layers.Conv2D(64, (5, 5), activation='relu'))
cnn.add(layers.Conv2D(64, (5, 5), activation='relu'))
cnn.add(layers.AveragePooling2D((2, 2)))
cnn.add(layers.Conv2D(128, (1, 1), activation='relu'))
cnn.add(layers.Conv2D(128, (1, 1), activation='relu'))
cnn.add(layers.AveragePooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(96, activation='relu'))
cnn.add(layers.Dense(10, activation='softmax'))

cnn.summary()

cnn.compile(optimizer="nadam",
            loss="categorical_crossentropy",
            metrics=["accuracy"])


# Pre trained network

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

mn2 = MobileNetV2(weights='imagenet',
                  input_shape=(200, 200, 3),
                  include_top=False)

mn2.trainable = False

mn2.summary()


# Data generator with and without data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

breedsDir = './breeds/'

breedsGen = ImageDataGenerator(preprocessing_function=preprocess_input)

breedsGenAug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=30,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode="nearest")

breedsTrainGen = breedsGen.flow_from_directory(directory=breedsDir + "train/",
                                               target_size=(200, 200),
                                               class_mode="categorical",
                                               batch_size=32)

breedsTestGen = breedsGen.flow_from_directory(directory=breedsDir + "test/",
                                              target_size=(200, 200),
                                              class_mode="categorical",
                                              batch_size=32)

breedsTrainGenAug = breedsGenAug.flow_from_directory(directory=breedsDir + "train/",
                                                     target_size=(200, 200),
                                                     class_mode="categorical",
                                                     batch_size=32)

speciesDir = './species/'

speciesGen = ImageDataGenerator(preprocessing_function=preprocess_input)

speciesGenAug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

speciesTrainGen = speciesGen.flow_from_directory(directory=speciesDir + "train/",
                                                 target_size=(200, 200),
                                                 class_mode="categorical",
                                                 batch_size=32)

speciesTestGen = speciesGen.flow_from_directory(directory=speciesDir + "test/",
                                                target_size=(200, 200),
                                                class_mode="categorical",
                                                batch_size=32)

speciesTrainGenAug = speciesGenAug.flow_from_directory(directory=speciesDir + "train/",
                                                       target_size=(200, 200),
                                                       class_mode="categorical",
                                                       batch_size=32)


Here we don't augment the data for the test set

If we were to use validation data, we wouldn't use data augmentation either